In [1]:
import sys
sys.path.append('..')

import decouple
import json
import os
import tqdm

import utils

NFS_DIR = decouple.config('NFS_PARENT_DIR')

In [2]:
data_config = utils.read_yaml('../config/pretrain/abcd_hibert_mlm-0.15.yaml')

with open(f'../project_dir/data/abcd/raw_data/abcd_v1.1.json', 'r') as f:
    raw_data = json.load(f)

In [3]:
flows = set()
subflows = set()
examples = raw_data['train']

for ex in examples:
    scenario = ex['scenario']
    flows.add(scenario['flow'])
    subflows.add(scenario['subflow'])

flow_to_idx = {flow: idx for idx, flow in enumerate(flows)}
subflow_to_idx = {subflow: idx for idx, subflow in enumerate(subflows)}

In [4]:
data_dir = '../project_dir/data/abcd/data'
os.makedirs(data_dir, exist_ok=True)

for split in data_config['config']['splits']:
    examples = raw_data[split]
    for dialog_idx, ex in tqdm.tqdm(enumerate(examples)):
        ex.pop('delexed')
        scenario = ex.pop('scenario')
        flow = scenario['flow']
        subflow = scenario['subflow']
        ex['example_id'] = ex.pop('convo_id')
        ex['flow'] = flow
        ex['subflow'] = subflow
        ex['flow_idx'] = flow_to_idx[flow]
        ex['subflow_idx'] = subflow_to_idx[subflow]
        ex['dialog_idx'] = dialog_idx
        dialog = []
        for turn_idx, turn in enumerate(ex.pop('original')):
            turn = {
                'party': turn[0],
                'turn': turn[1],
                'turn_idx': turn_idx,
            }
            dialog.append(turn)
        ex['dialogue'] = dialog

    path = data_config['path']['input_pattern_map'][split].format(nfs_dir=NFS_DIR)
    print(path)
    with open(path, 'w') as f:
        for example in examples:
            print(json.dumps(example), file=f)

8034it [00:00, 27262.42it/s]


/gscratch/ark/roylu/projects/conversation_pretraining/data/abcd/data/train.json


1004it [00:00, 38650.79it/s]


/gscratch/ark/roylu/projects/conversation_pretraining/data/abcd/data/dev.json


1004it [00:00, 39263.79it/s]

/gscratch/ark/roylu/projects/conversation_pretraining/data/abcd/data/test.json
